In [1]:
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils import data
import torchvision
from torchvision import datasets, models, transforms
from torchvision.models import alexnet

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Device:', device)

Device: cuda:0


In [3]:
os.chdir('C:/users/amod/pycharmprojects/cs184a')

In [4]:
data_dir = 'C:/Users/Amod/PycharmProjects/CS184a/hymenoptera_data'

In [5]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [6]:
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ('train', 'val')}
dataloaders = {x: data.DataLoader(image_datasets[x], batch_size=4,
                                  shuffle=(x =='train'), num_workers=4)
              for x in ('train', 'val')}
dataset_sizes = {x: len(image_datasets[x]) for x in ('train', 'val')}
class_names = image_datasets['train'].classes

In [7]:
num_epochs = 25
num_classes = 2
batch_size = 100
learning_rate = 0.001
rgb_means = [0.7024860505690291, 0.5462877872713974, 0.6964813026563755]
rgb_stds = [0.23890759190102304, 0.28214205984973445, 0.21625778862043862]

In [8]:
class AlmondNet(nn.Module):
    """
    A model that adapts the AlexNet CNN
    """
    
    def __init__(self, num_classes):
        super(AlmondNet, self).__init__()
        # Load the pretrained AlexNet model, 
        architecture = alexnet(pretrained=True)
        # copy architecture features and avg pool layer
        self.features = architecture.features
        self.avgpool = architecture.avgpool
        in_ftrs = architecture.classifier[1].in_features
        self.classifier = nn.Sequential(
            nn.Linear(in_features=in_ftrs, out_features=1024, bias=True),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5, inplace=False),
            nn.Linear(in_features=1024, out_features=1024, bias=True),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5, inplace=False),
            nn.Linear(in_features=1024, out_features=num_classes, bias=True)
        )
    
    def forward(self, x):
        """Does a forward pass on an image x."""
        out = self.features(x)
        out = self.avgpool(out)
        out = torch.flatten(out, 1)
        out = self.classifier(out)
        return out

model = AlmondNet(num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

KeyboardInterrupt: 

In [ ]:
print(model)

# Train on Hymenoptera Dataset

In [ ]:
def train(model, criterion, optimizer, scheduler, epochs=num_epochs):
    model.train()
    train_loader = dataloaders['train']
    since = time.time()
    num_steps = len(train_loader)
    for epoch in range(1, num_epochs+1):
        for i, (images, labels) in enumerate(train_loader, start=1):
            images = images.to(device)
            labels = labels.to(device)
            # Generate prediction and evaluate
            outputs = model(images)
            loss = criterion(outputs, labels)
            # Backpropagate loss and update weights
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            # Print epoch every 15 iterations
            if i % 15 == 0:
                print(f'Epoch [{epoch}/{num_epochs}], Step [{i}/{num_steps}], Loss: {loss.item():.6f}')
    # Print training time
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

In [ ]:
train(model, criterion, optimizer, scheduler=lr_scheduler, epochs=num_epochs)

In [ ]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in dataloaders['val']:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        predictions = torch.argmax(outputs, dim=1)

        total += labels.size(0)
        correct += (predictions == labels).sum().item()
    print(f'Test Accuracy of AlmondNet: {(100*(correct/total)):.6f}')